# Example 3-14: Performing an *IAU-2006/2000* Reduction
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, p. 221-223

This notebook demonstrates using the Celestial Intermediate Origin (CIO) theory to transform from the terrestrial (ITRF) coordinate system to the celestial (GCRF) coordinate system.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy

Import `os` and the relevant `valladopy` modules:

In [2]:
import os

import valladopy.constants as const
from valladopy.astro.time.data import iau06in, readxys
from valladopy.astro.time.frame_conversions import ecef2eci06
from valladopy.mathtime.julian_date import convtime

## Problem Definition
---

GIVEN: $\quad\overrightarrow{r}_{ITRF} = -1033.479383 \; \hat{I} \; + 7901.2952754 \; \hat{J} \; + 6380.3565958 \; \hat{K}$ km<br>
       $\quad\quad\quad\quad\overrightarrow{v}_{ITRF} = -3.22563652 \; \hat{I} \; - 2.87245145 \; \hat{J} \; + 5.531924446 \; \hat{K}$ km/s<br>
FIND:&emsp;$\quad\overrightarrow{r}_{GCRF},\ \overrightarrow{v}_{GCRF}$ on April 6, 2004, 07:51:28.386009 UTC

In [3]:
# ITRF (i.e. ECEF) state vectors
r_ecef = [-1033.479383, 7901.2952754, 6380.3565958]  # km
v_ecef = [-3.22563652, -2.87245145, 5.531924446]     # km/s
a_ecef = [0, 0, 0]                                   # dummy acceleration, km/s²

# Date
year = 2004
month = 4
day = 6
hour = 7
minute = 51
second = 28.386009
timezone = 0  # hours offset from UTC

## Solution
---

We first determine the time quantities prior to starting the problem.

Start by getting $\Delta AT$, from the *Astronomic Almanac* (or from USNO's historical list [here](https://maia.usno.navy.mil/ser7/tai-utc.dat)):

$$
\Delta AT = 32.0^{\text{S}}
$$

In [4]:
dat = 32  # seconds

We can find the EOP quantities from the IERS's EOPC04 [final data](https://datacenter.iers.org/data/latestVersion/finals.all.iau2000.txt). Note that these parameters are regularly updated, so we will use values from a past snapshot and convert some quantities from arcseconds to radians:

In [5]:
dut1 = -0.4399619                  # seconds
xp = -0.140682 * const.ARCSEC2RAD  # radians
yp = 0.333309 * const.ARCSEC2RAD   # radians
lod = 0.0015563                    # seconds
dx = -0.000205 * const.ARCSEC2RAD  # radians
dy = -0.000136 * const.ARCSEC2RAD  # radians

Although general practice interpolates these values with a cubic spline, the above snapshot represents non-interpolated values.

Next, convert UTC to various time systems using the `convtime` routine (see [Example 3-7](Example_3-7.ipynb) for more details on this process). We just need the Julian centuries of TT and the Julian date of UT1 for the final transformation:

In [6]:
_, _, jdut1, jdut1frac, _, _, _, ttt, *_ = convtime(year, month, day, hour, minute, second, timezone, dut1, dat)

print(f'Julian centuries of TT:\t{ttt}')

Julian centuries of TT:	0.04262363188899416


In order to convert the fixed vectors (ITRF/ECEF) to inertial vectors (GCRF/ECI), we need to obtain the matrices of coefficients for the IAU 2006 reduction calculations as well as the XYS data. The routines for these functions, `iau06in` for the matrices and `readxys` for the XYS data, require their respective data files to be present in a given data directory.

Replace the following data directory definition with your preferred location. We will use the relative path from this notebook, assuming the same structure as [repository](https://github.com/CelesTrak/fundamentals-of-astrodynamics) to reach the `datalib` directory. The file names are hardcoded for now but are planned to be flexible in the future — make sure these are included in your data directory.

In [7]:
data_dir = "../../../datalib/"  # relative data path

iau06arr = iau06in(data_dir)
iau06xysarr = readxys(data_dir)

print(f'IAU 06 coefficient matrices keys:\n{iau06arr.__dict__.keys()}\n')
print(f'IAU 06 XYS data keys:\n{iau06xysarr.__dict__.keys()}\n')

IAU 06 coefficient matrices keys:
dict_keys(['ax0', 'ax0i', 'ay0', 'ay0i', 'as0', 'as0i', 'agst', 'agsti', 'apn0', 'apn0i', 'apl0', 'apl0i', 'aapn0', 'aapn0i'])

IAU 06 XYS data keys:
dict_keys(['jd', 'jdf', 'x', 'y', 's', 'mjd_tt'])



**Algorithm 23** summarizes the process of transforming the state vectors between the fixed and inertial coordinate systems.

The precession-nutation matrix $PN$ is defined as:

$$
\begin{aligned}
\left[\mathbf{PN}\right] &= 
\left[\begin{array}{ccc}
1 - aX^2 & -aXY & X \\
-aXY & 1 - aY^2 & Y \\
-X & -Y & 1 - a(X^2 + Y^2)
\end{array}
\right] \text{ROT3}(s) \\
\\
a &= \frac{1}{1 + \cos(d)} \cong  \frac{1}{2} + \frac{1}{8} (X^2 + Y^2) \\
\\
d &= \arctan\left( \sqrt{ \frac{X^2 + Y^2}{1 - X^2 - Y^2} } \right) \\
\end{aligned} \\
$$

where $X$ and $Y$ are the coordinates of the Celestial Intermediate Pole (CIP), and $s$ is the position of the Celestial Intermediate Origin (CIO) on the moving equator of date.

The sidereal time rotation matrix $R$ is found with:

$$
[\mathbf{R}] = \text{ROT3}(-\theta_{ERA})
$$

where $\theta_{ERA}$ is the Earth Rotation Angle:

$$
\theta_{ERA} = 2\pi \ (0.779057273264 + 1.00273781191135448 \ (JD_{UT1} - 2,451,545.0))
$$

And the rotation matrix $W$ from ITRF to TIRS (to account for polar motion) is:

$$
[\mathbf{W}] = \text{ROT3}(-s') \cdot \text{ROT2}(x_p) \cdot \text{ROT1}(y_p)
$$

where $s'$ is the position of the Terrestrial Intermediate Origin (TIO) on the moving equator of the ITRF and $x_p$ and $y_p$ are the polar motion coordinates.

The GCRF (inertial) state vectors are then computed as:

$$
\begin{aligned}
\vec{r}_{GCRF} &= [\mathbf{PN}][\mathbf{R}][\mathbf{W}]\vec{r}_{ITRF} \\
\\
\vec{v}_{GCRF} &= [\mathbf{PN}][\mathbf{R}] \left\{ [\mathbf{W}]\vec{v}_{ITRF} + \vec{\omega}_\oplus \times \vec{r}_{TIRS} \right\}
\end{aligned}
$$

where $\vec{\omega}_\oplus$ is the Earth rotation vector (with $LOD$ as the length of day in seconds):

$$
\vec{\omega}_\oplus = \left[ 0,\ 0,\ 7.292115146706979 \times 10^{-5} \right]\left\{ 1 - \frac{LOD}{86400} \right\} \ \text{rad/s}
$$

and $\vec{r_{TIRS}}$ is simply:

$$
\vec{r_{TIRS}} = [\mathbf{W}]_{\text{ITRF-TIRS}} \ \ \vec{r_{ITRF}}
$$

All of this is handled internally by the `ecef2eci06` routine, which applies the IAU 2006 reduction using the provided EOP and XYS data:

In [8]:
r_eci, v_eci, _ = ecef2eci06(r_ecef, v_ecef, a_ecef, ttt, jdut1+jdut1frac, lod, xp, yp, iau06arr, iau06xysarr, dx, dy)

print('GCRF (ECI) state vectors:\n')
print(f'r_gcrf:\t{r_eci}\tkm')
print(f'v_gcrf:\t{v_eci}\t\tkm/s')

GCRF (ECI) state vectors:

r_gcrf:	[5102.50895292 6123.01139843 6378.13693439]	km
v_gcrf:	[-4.74322015  0.79053649  5.53375572]		km/s
